In [29]:
from scipy.stats import t
from scipy.special import comb
import numpy as np
import pandas as pd
import math

n = 5 #資產個數
n_simulation = 10000 #How many r.v.
n_steps = 20
K = 100
T = 0.5
r = 0.1
S = 95
q = 0.05
sigma = 0.5  # Common volatility
rho = 0.5   # Common correlation coefficient

def create_cov_matrix(n, sigma, rho):

    cov_matrix = np.full((n, n), rho)  
    np.fill_diagonal(cov_matrix, 1)    
    return sigma**2 * cov_matrix     




cov_matrix = create_cov_matrix(n, sigma, rho)


def Cholesky(cov_matrix):
    n = cov_matrix.shape[0]  # 從協方差矩陣獲取維度
    cholesky_decomp = np.zeros((n, n))  # 初始化 Cholesky 分解矩陣為零矩陣

    for i in range(n):
        for j in range(i, n):
            if i == j:  # 對角元素
                sum_ak2 = np.sum([cholesky_decomp[k, i] ** 2 for k in range(i)])
                cholesky_decomp[i, i] = np.sqrt(cov_matrix[i, i] - sum_ak2)
            else:  # 非對角元素
                sum_aki_akj = np.sum([cholesky_decomp[k, i] * cholesky_decomp[k, j] for k in range(i)])
                cholesky_decomp[i, j] = (cov_matrix[i, j] - sum_aki_akj) / cholesky_decomp[i, i]

    return cholesky_decomp



Monte-Carlo's simulation

In [30]:
#Antithetic variate approach and matching method
def anti_randon_variable(z):
    mean = np.zeros((n))
    stds = np.zeros((n))
    anti_z = -z
    randon_variable = np.vstack([z,anti_z])
    return randon_variable

def matching(randon_variable):   
    mean = np.mean(randon_variable,axis=0)
    stds = np.std(randon_variable,axis=0)
    z1 = (randon_variable-mean)/stds
    z2 = (randon_variable-mean)
    return z1,z2


def monte_carlo(S, K, T, r, q, n, n_simulation, n_steps):
    basic = []
    bonus1 = []
    bonus2 = []
    for _ in range(n_steps):
        #隨機變數處理
        z = np.random.randn(n_simulation//2,n)
        A = Cholesky(cov_matrix)*np.sqrt(T)
        z = anti_randon_variable(z)
        z1,z2 = matching(z)
        
        #bonus2隨機變數處理
        m = np.cov(z2,rowvar=False)
        m = Cholesky(m)
        m_inverse = np.linalg.inv(m)
        A_star = m_inverse.dot(A)

        #basic process
        rv = z.dot(A)
        ST = np.exp(np.log(S) +(r - q - 0.5 * sigma**2) * T + rv)
        max_st = np.max(ST,axis=1)
        payoff = np.exp(-r*T) * np.maximum((max_st - K), 0)
        average = np.mean(payoff)
        basic.append(average)

        #bonus1 process
        rv1 = z1.dot(A)
        ST1 =np.exp(np.log(S) +(r - q - 0.5 * sigma**2) * T + rv1)
        max_st1 = np.max(ST1,axis=1)
        payoff1 = np.exp(-r*T) * np.maximum((max_st1 - K), 0)
        average1 = np.mean(payoff1)
        bonus1.append(average1)

        #bonus2 process
        rv2 = z.dot(A_star)
        ST2 =np.exp(np.log(S) +(r - q - 0.5 * sigma**2) * T + rv2)
        max_st2 = np.max(ST2,axis=1)
        payoff2 = np.exp(-r*T) * np.maximum((max_st2 - K), 0)
        average2 = np.mean(payoff2)
        bonus2.append(average2)      

    ##output
    ##basic output
    basic_mean = np.mean(basic)
    basic_std = np.std(basic)
    upper = np.round(basic_mean + 2 * basic_std,4)
    lower = np.round(basic_mean - 2 * basic_std,4) 
    basic_CI_call = [lower,upper]

    ##bonus1 output
    bonus1_mean = np.mean(bonus1)
    bonus1_std = np.std(bonus1)
    upper = np.round(bonus1_mean + 2 * bonus1_std,4)
    lower = np.round(bonus1_mean - 2 * bonus1_std,4) 
    bonus1_CI_call = [lower,upper]

    ##bonus2 output
    bonus2_mean = np.mean(bonus2)
    bonus2_std = np.std(bonus2)
    upper = np.round(bonus2_mean + 2 * bonus2_std,4)
    lower = np.round(bonus2_mean - 2 * bonus2_std,4) 
    bonus2_CI_call = [lower,upper]

    return basic_mean,basic_std,basic_CI_call,bonus1_mean,bonus1_std,bonus1_CI_call,bonus2_mean,bonus2_std,bonus2_CI_call
        




basic_mean,basic_std,basic_CI_call,bonus1_mean,bonus1_std,bonus1_CI_call,bonus2_mean,bonus2_std,bonus2_CI_call = monte_carlo(S, K, T, r, q, n, n_simulation, n_steps)

print(f"Basic option value: {basic_mean:.4f}", f"std: {basic_std}", f"CI of call: {basic_CI_call}")
print(f"Bonus1 option value: {bonus1_mean:.4f}", f"std: {bonus1_std}",f"CI of call: {bonus1_CI_call}" )
print(f"Bonus2 option value: {bonus2_mean:.4f}", f"std: {bonus2_std}",f"CI of call: {bonus2_CI_call}" )

Basic option value: 30.3285 std: 0.14651715144740618 CI of call: [30.0355, 30.6215]
Bonus1 option value: 30.3638 std: 0.09307540846391797 CI of call: [30.1777, 30.55]
Bonus2 option value: 30.3730 std: 0.06383775815459751 CI of call: [30.2453, 30.5006]
